In [ ]:
from mplsoccer.pitch import Pitch, VerticalPitch
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from highlight_text import HighlightText, ax_text, fig_text
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
import matplotlib.patheffects as path_effects

In [ ]:
data = pd.read_csv("PL 23-24 WS Events.csv" , low_memory=False)
data['x'] = data['x']*1.2
data['y'] = data['y']*.8
data['endX'] = data['endX']*1.2
data['endY'] = data['endY']*.8

In [ ]:
info = pd.read_csv("T5 Leagues Mins 23-24.csv" , low_memory=False)

In [ ]:
replacement_dict = {
    'ENG-Premier League': 'Premier League',
    'ESP-La Liga': 'La Liga',
    'GER-Bundesliga': 'Bundesliga',
    'FRA-Ligue 1': 'Ligue 1',
    'ITA-Serie A': 'Serie A'
}

# Replace the values in the 'league' column
data['League'] = data['league'].replace(replacement_dict)
#data

In [ ]:
cmap_lst = ['#131313', '#24a8ff', '#0d4163'] 
cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 

In [ ]:
data['total_seconds'] = data['minute'] * 60 + data['second']

In [ ]:
# Define the function to calculate TTRP
def calculate_ttrp(data):
    # Initialize the new column with NaNs
    data['TTRP'] = np.nan
    
    # Define the preceding action types
    preceding_action_types = ['Aerial', 'BallRecovery', 'Tackle', 'TakeOn', 'Interception', 'Pass']
    
    # Filter data to get relevant actions
    relevant_actions = data[data['type'].isin(preceding_action_types)].copy()
    
    # Sort the data by gameId and total_seconds to ensure proper order
    relevant_actions.sort_values(by=['gameId', 'total_seconds'], inplace=True)
    
    # Create a shift column to get the previous action's time, type, and playerId within the same game
    relevant_actions['prev_total_seconds'] = relevant_actions.groupby('gameId')['total_seconds'].shift(1)
    relevant_actions['prev_type'] = relevant_actions.groupby('gameId')['type'].shift(1)
    relevant_actions['prev_playerId'] = relevant_actions.groupby('gameId')['playerId'].shift(1)
    
    # Filter only the passes
    passes = data[data['type'] == 'Pass'].copy()
    
    # Merge passes with relevant actions to get the preceding action details
    passes = passes.merge(
        relevant_actions[['gameId', 'playerId', 'total_seconds', 'prev_total_seconds', 'prev_type', 'prev_playerId']],
        how='left',
        left_on=['gameId', 'playerId', 'total_seconds'],
        right_on=['gameId', 'playerId', 'total_seconds']
    )
    
    # Calculate TTRP for valid preceding actions
    passes['TTRP'] = np.where(
        ((passes['prev_type'] == 'Pass') & (passes['prev_playerId'] != passes['playerId'])) |
        ((passes['prev_type'] != 'Pass') & (passes['prev_playerId'] == passes['playerId'])),
        passes['total_seconds'] - passes['prev_total_seconds'],
        np.nan
    )
    
    # Merge TTRP back to the original data
    data = data.merge(passes[['gameId', 'playerId', 'total_seconds', 'TTRP']], 
                      how='left', 
                      on=['gameId', 'playerId', 'total_seconds'], 
                      suffixes=('', '_y'))
    
    # Fill TTRP column
    data['TTRP'] = data['TTRP'].combine_first(data['TTRP_y'])
    data.drop(columns=['TTRP_y'], inplace=True)
    
    return data

# Apply the function
data = calculate_ttrp(data)




In [ ]:
data_passes = data[data['type']=='Pass']

In [ ]:
177/(2161/90)

In [ ]:
# Assume the data DataFrame has already been processed with the calculate_ttrp function
# Filter for successful passes
successful_passes = data[(data['type'] == 'Pass') & (data['outcomeType'] == 'Successful')]

# Filter rows with valid TTRP values
valid_ttrp_data = successful_passes.dropna(subset=['TTRP'])

# Group by playerId, player, and team to calculate the required metrics
ttrp_df = valid_ttrp_data.groupby(['playerId', 'player', 'team', 'League']).agg(
    total_ttrp=('TTRP', 'sum'),
    true_passes=('TTRP', 'count')
).reset_index()

# Ensure playerId is the first column
ttrp_df = ttrp_df[['playerId', 'player', 'team', 'total_ttrp', 'true_passes', 'League']]

# Print the resulting DataFrame to verify
ttrp_df['TTRP per Pass'] = ttrp_df['total_ttrp']/ttrp_df['true_passes']


all_passes = data[data['type'] == 'Pass']

# Group by playerId, player, and team to calculate the successful and unsuccessful passes
pass_accuracy_df = all_passes.groupby(['playerId', 'player', 'team']).agg(
    successful_passes=('outcomeType', lambda x: (x == 'Successful').sum()),
    total_passes=('outcomeType', 'count')
).reset_index()

# Calculate the passing accuracy
pass_accuracy_df['passing_accuracy'] = (pass_accuracy_df['successful_passes'] / pass_accuracy_df['total_passes']) * 100

# Merge the passing accuracy with the ttrp_df
ttrp_df = ttrp_df.merge(pass_accuracy_df[['playerId', 'passing_accuracy']], on=['playerId'], how='left')


#ttrp_df['player'] = ttrp_df['player'].str.strip().str.upper()
#info['player'] = info['player'].str.strip().str.upper()
#ttrp_df = pd.merge(ttrp_df, info, on=['player', 'League'], how='inner')

#ttrp_df = ttrp_df[['player', 'team', 'Mins', 'Age', 'Position', 'total_ttrp', 'true_passes', 'TTRP per Pass']]

#disp = ttrp_df[ttrp_df['true_passes'] > 150].sort_values('TTRP per Pass', ascending=True)
#disp = disp.head(20)
#disp

disp2 = ttrp_df[(ttrp_df['team'] == 'Everton') & (ttrp_df['true_passes'] > 100)].sort_values('TTRP per Pass', ascending=True)
disp2 = disp2.head(18)
disp2
#ttrp_df

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

# Assuming df_plot is already defined and contains the required columns

# Create the plot with the specified background color
fig, ax = plt.subplots(figsize=(18, 12))
fig.patch.set_facecolor('#f8f8ee')
ax.set_facecolor('#f8f8ee')

bars = ax.barh(disp2['player'], disp2['TTRP per Pass'], color='#877ae8', height=0.65)

# Set the title and labels with custom color
ax.text(0.0, 1.1, 'Average Time Taken to Release the Ball (in Seconds)\nEuros | Min. 150 True Passes | 2024', font='Arial Rounded MT Bold', fontsize=28,
        color='#877ae8', ha='left', va='top', transform=ax.transAxes)

ax.set_xlabel('Ball Retention %', font='Arial Rounded MT Bold', fontsize=12, color='#877ae8')

# Add a line below the title
#ax.axhline(y=-1.0, color='white', linewidth=1.5, alpha=0.5)  

# Set the font properties for player names and percentage labels
font_prop = FontProperties(family='Arial Rounded MT Bold', size=18)

# Add annotations with custom color and font properties
for bar in bars:
    ax.text(bar.get_width() + 0.05, bar.get_y() + bar.get_height() / 2,
            f'{bar.get_width():.1f}', 
            va='center', ha='left', fontsize=18, color='#877ae8', fontproperties=font_prop)

# Invert the y-axis to have the player with the highest retention at the top
ax.invert_yaxis()

# Remove the x-axis numbers and ticks, but keep the player names on the y-axis
ax.tick_params(bottom=False, labelbottom=False)
ax.xaxis.set_visible(False)  # This removes the x-axis numbers

# Customize the y-axis tick labels
for tick in ax.get_yticklabels():
    tick.set_fontproperties(font_prop)
    tick.set_color('#877ae8')

# Remove the spines
for spine in ax.spines.values():
    spine.set_visible(False)

# Add text for author and data source on the bottom right corner
ax.text(0.995, -0.02, 'Made by: @pranav_m28 | Data: Opta', font='Arial Rounded MT Bold',color='#877ae8', fontsize=12,
        ha='right', va='bottom', transform=ax.transAxes)


#plt.savefig("Everton TTRP.png" , dpi=300, bbox_inches='tight')

# Display the plot
plt.show()

In [ ]:
scatter = ttrp_df[ttrp_df['true_passes'] >= 100]

# Create the plot with the specified background color
fig, ax = plt.subplots(figsize=(18, 12))
fig.patch.set_facecolor('#f8f8ee')
ax.set_facecolor('#f8f8ee')

plot = plt.scatter(scatter['TTRP per Pass'], scatter['passing_accuracy'], color='#877ae8', s=200, alpha=0.6, linewidth=1.5)

# Set the title and labels with custom color
ax.text(0.0, 1.12, 'Passing Speed vs Reliability\nEuropean Championship | Min. 100 True Passes | 2024', 
        font='Arial Rounded MT Bold', fontsize=28, color='#877ae8', ha='left', va='top', transform=ax.transAxes)

ax.set_xlabel('Average Time Taken to Release a Pass', font='Arial Rounded MT Bold', fontsize=18, color='#877ae8', labelpad=24)
ax.set_ylabel('Passing Accuracy %', font='Arial Rounded MT Bold', fontsize=18, color='#877ae8', labelpad=24)

# Set the font properties for player names and percentage labels
font_prop = FontProperties(family='Arial Rounded MT Bold', size=18)

ax.invert_xaxis()

# Add player names below each scatter point
#for i, row in filter_actions.iterrows():
 #   if row['ball_won_hs_p90'] > 1.5 and row['tack+int_hs_p90'] > 0.5:
  #      ax.text(row['ball_won_hs_p90'], row['tack+int_hs_p90'], row['player'], 
   #             ha='center', va='center', fontsize=12, color='#877ae8', fontproperties=font_prop)

# Remove the x-axis numbers and ticks, but keep the player names on the y-axis
ax.tick_params(bottom=True, labelbottom=True)
ax.xaxis.set_visible(True)  # This removes the x-axis numbers

# Customize the y-axis tick labels
for tick in ax.get_yticklabels():
    tick.set_fontproperties(font_prop)
    tick.set_color('#877ae8')
    
# Customize the y-axis tick labels
for tick in ax.get_xticklabels():
    tick.set_fontproperties(font_prop)
    tick.set_color('#877ae8')

# Remove the spines
for spine in ax.spines.values():
    spine.set_visible(False)

# Remove only the left spine
ax.spines['left'].set_visible(False)

# Add text for author and data source on the bottom right corner
ax.text(0.995, -0.12, 'Made by: @pranav_m28 | Data: Opta', font='Arial Rounded MT Bold', color='#877ae8', fontsize=12,
        ha='right', va='bottom', transform=ax.transAxes)


plt.text(2.50, 92.46, 'Bellingham', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(2.28, 90.33, 'Andrich', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(2.52, 85.48, 'Kante', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(2.70, 88.35, 'Hjulmand', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(2.65, 90.79, 'Wirtz', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(2.71, 86.40, 'Onana', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(2.67, 94.28, 'Lobotka', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(2.77, 83.91, 'Orban', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(6.39, 90.15, 'Akanji', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(5.87, 91.68, 'Dias', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(4.99, 90.57, 'Van Dijk', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(4.62, 91.97, 'Bastoni', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(4.69, 88.40, 'Gvardiol', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(4.29, 90.33, 'Stones', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(5.15, 93.46, 'Skriniar', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(5.84, 86.31, 'Mitaj', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(3.29, 96.63, 'Christensen', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(3.88, 95.94, 'Saliba', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(3.54, 94.79, 'Tchouameni', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(3.05, 93.45, 'Rice', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(4.14, 94.49, 'Sutalo', font='Arial Rounded MT Bold', size=15, color='#877ae8')
plt.text(2.81, 89.13, 'Højberg', font='Arial Rounded MT Bold', size=15, color='#877ae8')


plt.text(2.60, 77, 'Quick but Unreliable', font='Arial Rounded MT Bold', size=20, color='#33b864')

plt.text(6.40, 97.5, 'Slow but Reliable', font='Arial Rounded MT Bold', size=20, color='#33b864')

plt.scatter(2.849, 93.032, color='#FF5959', s=200, alpha=0.6, lw=1.5)

#plt.savefig("Cala.png" , dpi=300, bbox_inches='tight')

# Display the plot
plt.show()


In [ ]:
scatter.sort_values('TTRP per Pass', ascending=True).head(20)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
# Create the plot with the specified background color
fig, ax = plt.subplots(figsize=(18, 12))
fig.patch.set_facecolor('#f8f8ee')
ax.set_facecolor('#f8f8ee')

plot = plt.scatter(scatter['TTRP per Pass'], scatter['passing_accuracy'], color='#877ae8', s=200, alpha=0.6, linewidth=1.5)

# Set the title and labels with custom color
ax.text(0.0, 1.12, 'Passing Speed vs Reliability\nEuropean Championship | Min. 100 True Passes | 2024', 
        font='Arial Rounded MT Bold', fontsize=28, color='#877ae8', ha='left', va='top', transform=ax.transAxes)

ax.set_xlabel('Average Time Taken to Release a Pass', font='Arial Rounded MT Bold', fontsize=18, color='#877ae8', labelpad=24)
ax.set_ylabel('Passing Accuracy %', font='Arial Rounded MT Bold', fontsize=18, color='#877ae8', labelpad=24)

# Set the font properties for player names and percentage labels
font_prop = FontProperties(family='Arial Rounded MT Bold', size=18)

ax.invert_xaxis()

# Add player names below each scatter point
#for i, row in filter_actions.iterrows():
 #   if row['ball_won_hs_p90'] > 1.5 and row['tack+int_hs_p90'] > 0.5:
  #      ax.text(row['ball_won_hs_p90'], row['tack+int_hs_p90'], row['player'], 
   #             ha='center', va='center', fontsize=12, color='#877ae8', fontproperties=font_prop)

# Remove the x-axis numbers and ticks, but keep the player names on the y-axis
ax.tick_params(bottom=True, labelbottom=True)
ax.xaxis.set_visible(True)  # This removes the x-axis numbers

# Customize the y-axis tick labels
for tick in ax.get_yticklabels():
    tick.set_fontproperties(font_prop)
    tick.set_color('#877ae8')
    
# Customize the y-axis tick labels
for tick in ax.get_xticklabels():
    tick.set_fontproperties(font_prop)
    tick.set_color('#877ae8')

# Remove the spines
for spine in ax.spines.values():
    spine.set_visible(False)

# Remove only the left spine
ax.spines['left'].set_visible(False)

# Add text for author and data source on the bottom right corner
ax.text(0.995, -0.12, 'Made by: @pranav_m28 | Data: Opta', font='Arial Rounded MT Bold', color='#877ae8', fontsize=12,
        ha='right', va='bottom', transform=ax.transAxes)


#plt.text(2.50, 92.46, 'Bellingham', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(2.28, 90.33, 'Andrich', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(2.52, 85.48, 'Kante', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(2.70, 88.35, 'Hjulmand', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(2.65, 90.79, 'Wirtz', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(2.71, 86.40, 'Onana', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(2.67, 94.28, 'Lobotka', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(2.77, 83.91, 'Orban', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(6.39, 90.15, 'Akanji', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(5.87, 91.68, 'Dias', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(4.99, 90.57, 'Van Dijk', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(4.62, 91.97, 'Bastoni', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(4.69, 88.40, 'Gvardiol', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(4.29, 90.33, 'Stones', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(5.15, 93.46, 'Skriniar', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(5.84, 86.31, 'Mitaj', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(3.29, 96.63, 'Christensen', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(3.88, 95.94, 'Saliba', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(3.54, 94.79, 'Tchouameni', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(3.05, 93.45, 'Rice', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(4.14, 94.49, 'Sutalo', font='Arial Rounded MT Bold', size=15, color='#877ae8')
#plt.text(2.81, 89.13, 'Højberg', font='Arial Rounded MT Bold', size=15, color='#877ae8')


plt.text(2.60, 77, 'Quick but Unreliable', font='Arial Rounded MT Bold', size=20, color='#33b864')

plt.text(6.40, 97.5, 'Slow but Reliable', font='Arial Rounded MT Bold', size=20, color='#33b864')

plt.scatter(2.849, 93.032, color='#FF5959', s=200, alpha=0.6, lw=1.5)

#plt.savefig("Cala.png" , dpi=300, bbox_inches='tight')

# Display the plot
plt.show()